# Import Libraries

In [1]:
import google.generativeai as genai  
from dotenv import load_dotenv
import os

c:\Users\ghost\miniconda3\envs\de_en\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)  

In [3]:

model = genai.GenerativeModel("gemini-2.0-flash")


In [29]:
prompt_template = '''
Create a natural-sounding conversation between two or more people about {}.
Use realistic, everyday language and natural dialogue flow.
Choose random character names. Only provide the conversation—no explanations or descriptions needed.
Write the character's name first, followed by their dialogue.
Do not include asterisks or any narration.
'''


In [42]:

topic = "about thier university" 
prompt = prompt_template.format(topic)


In [43]:

response = model.generate_content(prompt)
print(response.text)


Liam: Hey Maya, you heading to Professor Chen's lecture?

Maya: Yeah, just grabbing a coffee first. You survived last night's party, I see.

Liam: Barely. Sarah threw an absolute rager. Did you make it out?

Maya: I was studying for Chen's midterm, sadly. How was it? Worth missing the quadratic formula?

Liam: Totally worth it. Until about 2 am. Then I was just questioning all my life choices. Hey, speaking of questioning…

Chloe: Liam! Maya! What's up?

Liam: Hey Chloe. Just asking Maya if she was at Sarah's party last night. You were there, right?

Chloe: Oh god, yes. Way too much jungle juice. I think I accidentally insulted the president of the Accounting Society.

Maya: Oh no! What did you say?

Chloe: Something about his calculator being outdated. I don't even know. I blacked out most of it. Anyway, what are you guys talking about? Besides my impending social doom?

Liam: I was gonna ask Maya what she thought about switching my major to Environmental Science. I’m having second th

# TTS

In [44]:
import re
import time
import random
import os
import wave
import contextlib
from TTS.api import TTS


In [45]:

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint_path = "tts_models/en/vctk/vits"
tts = TTS(model_name=checkpoint_path,progress_bar=True).to(device)

In [46]:
voice_pool = [
    "p225", "p226", "p227", "p228", "p229", "p230", "p231", "p232", "p233", "p234",
    "p236", "p237", "p238", "p239", "p240", "p241", "p243", "p244", "p245", "p246",
    "p247", "p248", "p249", "p250", "p251", "p252", "p253", "p254", "p255", "p256",
    "p257", "p258", "p259", "p260", "p261", "p262", "p263", "p264", "p265", "p266",
    "p267", "p268", "p269", "p270", "p271", "p272", "p273", "p274", "p275", "p276",
    "p277", "p278", "p279", "p280", "p281", "p282", "p283", "p284", "p285", "p286",
    "p287", "p288", "p292", "p293", "p294", "p295", "p297", "p298", "p299", "p300",
    "p301", "p302", "p303", "p304", "p305", "p306", "p307", "p308", "p310", "p311",
    "p312", "p313", "p314", "p316", "p317", "p318", "p323", "p326", "p329", "p330",
    "p333", "p334", "p335", "p336", "p339", "p340", "p341", "p343", "p345", "p347",
    "p351", "p360", "p361", "p362", "p363", "p364", "p374", "p376"
]


In [47]:
import random

In [48]:
dialogue = response.text.strip()

# Parse lines like "Maria: Hi there"
lines = [line.strip() for line in dialogue.split('\n') if ':' in line]
speakers = list({line.split(':')[0].strip() for line in lines})
random.shuffle(voice_pool)
speaker_voice_map = {speaker: voice_pool[i] for i, speaker in enumerate(speakers)}

# Create directory for temporary audio
os.makedirs("temp_audio", exist_ok=True)
temp_files = []

# Generate TTS per line
for idx, line in enumerate(lines):
    speaker, text = line.split(':', 1)
    speaker = speaker.strip()
    text = text.strip()
    voice = speaker_voice_map[speaker]
    
    file_path = f"temp_audio/line_{idx:03}.wav"
    tts.tts_to_file(text=text, speaker=voice, file_path=file_path)
    temp_files.append(file_path)

# Concatenate WAV files using `wave`
output_file = "final_output.wav"

with wave.open(output_file, 'wb') as out_wav:
    with wave.open(temp_files[0], 'rb') as first:
        out_wav.setparams(first.getparams())
    
    for fname in temp_files:
        with wave.open(fname, 'rb') as w:
            frames = w.readframes(w.getnframes())
            out_wav.writeframes(frames)

print("✅ Final audio saved to:", output_file)

✅ Final audio saved to: final_output.wav
